In [61]:
import time
import pickle
from collections import defaultdict

import numpy as np
import pandas as pd

In [37]:
!ls

Partition6467LinkData.csv    main.ipynb
Partition6467ProbePoints.csv readme - mapmatching.txt


In [38]:
link_data_column_names = ['linkPVID', 'refNodeID', 'nrefNodeID', 'length', 'functionalClass', 'directionOfTravel', \
                'speedCategory', 'fromRefSpeedLimit', 'toRefSpeedLimit', 'fromRefNumLanes', 'toRefNumLanes', \
                'multiDigitized', 'urban', 'timeZone', 'shapeInfo', 'curvatureInfo', 'slopeInfo']

In [39]:
probe_data_column_names = ['sampleID', 'dateTime', 'sourceCode', 'latitude',\
                           'longitude', 'altitude', 'speed', 'heading']

In [40]:
df_link_data = pd.read_csv('Partition6467LinkData.csv', names=link_data_column_names)
df_link_data.head(10)

,linkPVID,refNodeID,nrefNodeID,length,functionalClass,directionOfTravel,speedCategory,fromRefSpeedLimit,toRefSpeedLimit,fromRefNumLanes,toRefNumLanes,multiDigitized,urban,timeZone,shapeInfo,curvatureInfo,slopeInfo
0,62007637,162844982,162809070,335.04,5,B,7,30,30,0,0,F,T,0.0,51.4965800/9.3862299/|51.4994700/9.3848799/,NaN,NaN
1,567329767,162844982,162981512,134.56,5,B,7,0,0,0,0,F,T,0.0,51.4965800/9.3862299/|51.4966899/9.3867100/|51...,NaN,NaN
2,62007648,162877732,162844982,97.01,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4965800/9.3862299/,NaN,NaN
3,78670326,162877732,163152693,314.84,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4990000/9.3836099/,NaN,NaN
4,51881672,174713859,174587951,110.17,3,B,6,50,50,2,2,F,T,0.0,53.0643099/8.7903400/45.79|53.0650299/8.791470...,NaN,0.00/-0.090|110.17/0.062
5,51881767,174587951,174385294,212.54,3,B,7,50,50,2,2,F,T,0.0,53.0650299/8.7914700/45.74|53.0657462/8.792644...,111.93/-0.000026,0.00/0.062|111.93/0.170|212.54/0.081
6,51881768,174385294,174522149,186.28,3,B,7,50,50,2,2,F,T,0.0,53.0663900/8.7937000/46.25|53.0676300/8.795570...,NaN,0.00/0.081|186.28/-0.114
7,51881825,174522149,174535472,140.13,3,B,6,50,50,2,2,F,T,0.0,53.0676300/8.7955700/46.13|53.0685500/8.797000...,NaN,0.00/-0.114|140.13/-0.120
8,51881938,174535472,174496573,85.54,3,B,6,50,50,2,2,F,T,0.0,53.0685500/8.7970000/45.83|53.0688210/8.797456...,42.93/-0.000346,0.00/-0.120|42.93/0.162|85.54/0.328
9,811768915,174496573,1831479441,59.45,3,F,6,50,0,2,0,T,T,0.0,53.0690900/8.7979099/46.04|53.0692003/8.798316...,29.82/0.001684,0.00/0.228|29.82/0.076|59.45/0.222


In [41]:
def extract_coordinates(string):
    entries = string.split('|')
    (lat1, lon1, ele1) = entries[0].split('/')
    (lat2, lon2, ele2) = entries[1].split('/')
    return (float(lat1), float(lon1), ele1), (float(lat2), float(lon2), ele2)

In [42]:
coor_info = extract_coordinates(df_link_data['shapeInfo'][1])
print(coor_info)
coor_info[0]

((51.49658, 9.3862299, ''), (51.4966899, 9.38671, ''))


(51.49658, 9.3862299, '')

In [43]:
float(coor_info[0][0])

51.49658

In [44]:
df_link_data.shape

(200089, 17)

In [45]:
df_probe_data = pd.read_csv('Partition6467ProbePoints.csv', names=probe_data_column_names)
df_probe_data.head(10)

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading
0,3496,6/12/2009 6:12:49 AM,13,51.496868,9.386022,200,23,339
1,3496,6/12/2009 6:12:54 AM,13,51.496682,9.386157,200,10,129
2,3496,6/12/2009 6:12:59 AM,13,51.496705,9.386422,201,21,60
3,3496,6/12/2009 6:13:04 AM,13,51.496749,9.386840,201,0,360
4,3496,6/12/2009 6:13:09 AM,13,51.496864,9.387294,199,0,360
5,3496,6/12/2009 6:13:15 AM,13,51.496930,9.387716,198,5,89
6,3496,6/12/2009 6:13:20 AM,13,51.496957,9.387794,198,1,288
7,3496,6/12/2009 6:13:25 AM,13,51.496952,9.387805,197,0,310
8,3496,6/12/2009 6:13:30 AM,13,51.496949,9.387818,196,0,274
9,3496,6/12/2009 6:13:35 AM,13,51.496944,9.387840,196,0,226


In [46]:
df_probe_data.shape

(3375745, 8)

In [47]:
# df_probe_data['sampleID'].value_counts()

In [48]:
df_probe = df_probe_data[df_probe_data['sampleID']==3496]
df_probe.head(10)

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading
0,3496,6/12/2009 6:12:49 AM,13,51.496868,9.386022,200,23,339
1,3496,6/12/2009 6:12:54 AM,13,51.496682,9.386157,200,10,129
2,3496,6/12/2009 6:12:59 AM,13,51.496705,9.386422,201,21,60
3,3496,6/12/2009 6:13:04 AM,13,51.496749,9.386840,201,0,360
4,3496,6/12/2009 6:13:09 AM,13,51.496864,9.387294,199,0,360
5,3496,6/12/2009 6:13:15 AM,13,51.496930,9.387716,198,5,89
6,3496,6/12/2009 6:13:20 AM,13,51.496957,9.387794,198,1,288
7,3496,6/12/2009 6:13:25 AM,13,51.496952,9.387805,197,0,310
8,3496,6/12/2009 6:13:30 AM,13,51.496949,9.387818,196,0,274
9,3496,6/12/2009 6:13:35 AM,13,51.496944,9.387840,196,0,226


In [49]:
df_link_data.iloc[0, :]['shapeInfo']

'51.4965800/9.3862299/|51.4994700/9.3848799/'

In [50]:
# reference: https://en.wikipedia.org/wiki/Haversine_formula

def haversine(lat1, lon1, lat2, lon2):
    '''The haversine formula determines the great-circle distance 
    between two points on a sphere given their longitudes and latitudes.'''
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = np.radians([lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.square(np.sin(dlat/2)) + np.cos(lat1)*np.cos(lat2)*np.square(np.sin(dlon/2))
    c = 2 * np.arcsin(np.sqrt(a)) 
    R = 6371 # radius of earth in kilometers; use 3956 for miles
    return R * c

In [51]:
# reference: https://en.wikipedia.org/wiki/Heron%27s_formula

def heron(a, b, c):
    '''use Heron's formula to determine height'''
    s = (a+b+c)/2
    if s*(s-a)*(s-b)*(s-c) < 0:
        print('a:', a)
        print('b:', b)
        print('c:', c)
        print('pre sqrt:', s*(s-a)*(s-b)*(s-c))
        return None
    area = np.sqrt(s*(s-a)*(s-b)*(s-c))
    return 2*area/c

In [52]:
def correct_length(a, b, c):
    '''correct lengthes of triangle sides'''
    if a + b >= c:
        return a, b, c
    else:
        while a + b <= c:
            a += 1
            b += 1
        return a, b, c

In [53]:
len(set(df_link_data['linkPVID'])) == len(df_link_data['linkPVID'])

True

In [ ]:
start_time = time.time()

road_altitudes_dict = defaultdict(list)

for j in range(len(df_probe_data)):
    print('j:', j)
# for j in range(1):
    probe = df_probe_data.iloc[j,:]
    probe_lat = probe['latitude']
    probe_lon = probe['longitude']
    
    road_id = None
    min_dist = 100000
    
    # find the closest road to probe
    for i in range(len(df_link_data)):
        road = df_link_data.iloc[i, :]
        coor = extract_coordinates(road['shapeInfo'])
        a = haversine(coor[0][0], coor[0][1], probe_lat, probe_lon) * 1000
        b = haversine(coor[1][0], coor[1][1], probe_lat, probe_lon) * 1000
        c = road['length']
        a, b, c = correct_length(a, b, c)
        h = heron(a, b, c)
        if h is None:
            continue
        # update closest road
        if h < min_dist:
            road_id = road['linkPVID']
            min_dist = h

    road_altitudes_dict[road_id].append(probe['altitude'])

print('finished in {:.3}s'.format(time.time()-start_time))

a: 107906.64299266272
b: 107896.60060313187
c: 10.04
pre sqrt: -139676703.43357047


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in sqrt
  # This is added back by InteractiveShellApp.init_path()


a: 107938.18713539996
b: 107928.14474490346
c: 10.04
pre sqrt: -139814868.35959122
a: 107974.4864945928
b: 107964.44410381347
c: 10.04
pre sqrt: -139925482.63503638
a: 108013.79652278143
b: 108003.75413261751
c: 10.04
pre sqrt: -139991342.47890514
a: 108040.4402100407
b: 108030.39782042368
c: 10.04
pre sqrt: -140028366.49884766
a: 108040.7078123507
b: 108030.6654228097
c: 10.04
pre sqrt: -140024603.3151877
a: 108039.93235793395
b: 108029.88996845491
c: 10.04
pre sqrt: -140018963.6098964
a: 108040.40137936344
b: 108030.35898989189
c: 10.04
pre sqrt: -140019739.30895326
a: 108040.77462596432
b: 108030.73223648616
c: 10.04
pre sqrt: -140021093.99108914
a: 108043.24781879068
b: 108033.2054292305
c: 10.04
pre sqrt: -140032311.6571025
a: 108042.46184322528
b: 108032.41945371332
c: 10.04
pre sqrt: -140027449.54095078
a: 108049.5347069713
b: 108039.4923177523
c: 10.04
pre sqrt: -140028612.48662135
a: 108098.08053712599
b: 108088.03814808678
c: 10.04
pre sqrt: -140143925.431803
a: 108153.250850

a: 761.9816627288479
b: 757.390506287216
c: 4.59
pre sqrt: -1531.8826354615412
a: 106709.8501846317
b: 106699.80803673355
c: 10.04
pre sqrt: -122781235.51889345
a: 818.0476161606995
b: 813.4567940180773
c: 4.59
pre sqrt: -1255.6882257955529
a: 106740.26763282389
b: 106730.22545473608
c: 10.04
pre sqrt: -124578162.16445334
a: 106777.00489618775
b: 106766.96269502239
c: 10.04
pre sqrt: -125984932.96339755
a: 106803.63850973222
b: 106793.59628987452
c: 10.04
pre sqrt: -127118313.62255658
a: 106832.49569209521
b: 106822.45345228257
c: 10.04
pre sqrt: -128330465.45856397
a: 106864.03741448837
b: 106853.9951518741
c: 10.04
pre sqrt: -129713600.20549588
a: 106898.18071164613
b: 106888.13842720112
c: 10.04
pre sqrt: -131048984.35456626
a: 106930.86087836312
b: 106920.81857625135
c: 10.04
pre sqrt: -132143330.300962
a: 106964.64713782199
b: 106954.60482057947
c: 10.04
pre sqrt: -133096019.5221556
a: 107001.70223175973
b: 106991.65990003844
c: 10.04
pre sqrt: -134020549.03355062
a: 107043.416616

In [ ]:
with open('roads.pickle', 'wb') as file:
    pickle.dump(road_altitudes_dict, file, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
lon1 = -103.548851
lat1 = 32.0004311
lon2 = -103.6041946
lat2 = 33.374939

print(haversine(lat1, lon1, lat2, lon2))

152.92602752348105
